**This notebook is an exercise in the [Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/coordinate-reference-systems).**

---


# Introduction

You are a bird conservation expert and want to understand migration patterns of purple martins.  In your research, you discover that these birds typically spend the summer breeding season in the eastern United States, and then migrate to South America for the winter.  But since this bird is under threat of endangerment, you'd like to take a closer look at the locations that these birds are more likely to visit.

<center>
<img src="https://i.imgur.com/qQcS0KM.png" width="1000"><br/>
</center>

There are several [protected areas](https://www.iucn.org/theme/protected-areas/about) in South America, which operate under special regulations to ensure that species that migrate (or live) there have the best opportunity to thrive.  You'd like to know if purple martins tend to visit these areas.  To answer this question, you'll use some recently collected data that tracks the year-round location of eleven different birds.

Before you get started, run the code cell below to set everything up.

In [ ]:
import pandas as pd
import geopandas as gpd

from shapely.geometry import LineString

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex2 import *

# Exercises

### 1) Load the data.

다음 코드 셀(변경 없이)을 실행하여 GPS 데이터를 pandas DataFrame `birds_df`에 로드합니다.  

In [ ]:
# Load the data and print the first 5 rows
birds_df = pd.read_csv("../input/geospatial-learn-course-data/purple_martin.csv", parse_dates=['timestamp'])
print("There are {} different birds in the dataset.".format(birds_df["tag-local-identifier"].nunique()))
birds_df.head()

데이터 세트에는 11마리의 새가 있으며 각 새는 "tag-local-identifier" 열의 고유한 값으로 식별됩니다. 각 새는 일년 중 다른시기에 수집 된 여러 측정 값을 가지고 있습니다.

다음 코드 셀을 사용하여 GeoDataFrame '새'를 만듭니다.
- `birds`에는 (경도, 위도) 위치가 있는 Point 개체를 포함하는 "기하학" 열과 함께 `birds_df`의 모든 열이 있어야 합니다.
- `birds`의 CRS를 `{'init': 'epsg:4326'}`으로 설정합니다.

In [ ]:
# Your code here: Create the GeoDataFrame
birds =birds = gpd.GeoDataFrame(birds_df, geometry=gpd.points_from_xy(birds_df["location-long"], birds_df["location-lat"]))

# Your code here: Set the CRS to {'init': 'epsg:4326'}
birds.crs = birds.crs = {'init' :'epsg:4326'}


# Check your answer
q_1.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Plot the data.

 다음으로 GeoPandas에서 `'naturalearth_lowres'' 데이터 세트를 로드하고 `americas`를 미주(북미와 남미 모두)의 모든 국가 경계를 포함하는 GeoDataFrame으로 설정합니다. 변경 없이 다음 코드 셀을 실행합니다.

In [ ]:
# Load a GeoDataFrame with country boundaries in North/South America, print the first 5 rows
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
americas = world.loc[world['continent'].isin(['North America', 'South America'])]
americas.head()

다음 코드 셀을 사용하여 (1) `americas` GeoDataFrame의 국가 경계와 (2) `birds_gdf` GeoDataFrame의 모든 점을 모두 표시하는 단일 플롯을 만듭니다.

여기에서 특별한 스타일링에 대해 걱정하지 마십시오. 모든 데이터가 제대로 로드되었는지 신속하게 확인하기 위해 예비 플롯을 생성하기만 하면 됩니다. 특히 새를 구분하기 위해 포인트를 색으로 구분할 필요가 없고, 시작점과 끝점을 구분할 필요가 없습니다. 연습의 다음 부분에서 그렇게 할 것입니다.

In [ ]:
# Your code here
ax = americas.plot(figsize=(10,10), color='white', linestyle=':', edgecolor='red')
birds.plot(ax=ax, markersize=10)

# Uncomment to see a hint
#q_2.hint()

In [ ]:
# Get credit for your work after you have created a map
#q_2.check()

# Uncomment to see our solution (your code may look different!)
#q_2.solution()

### 3) Where does each bird start and end its journey? (Part 1)

이제 각 새의 경로를 더 자세히 볼 준비가 되었습니다. 다음 코드 셀을 실행하여 두 개의 GeoDataFrame을 만듭니다.
- `path_gdf`에는 각 새의 경로를 표시하는 LineString 개체가 포함되어 있습니다. 'LineString()' 메서드를 사용하여 Point 개체 목록에서 LineString 개체를 만듭니다.
- `start_gdf`는 각 새의 시작 지점을 포함합니다.

In [ ]:
# GeoDataFrame showing path for each bird
path_df = birds.groupby("tag-local-identifier")['geometry'].apply(list).apply(lambda x: LineString(x)).reset_index()
path_gdf = gpd.GeoDataFrame(path_df, geometry=path_df.geometry)
path_gdf.crs = {'init' :'epsg:4326'}

# GeoDataFrame showing starting point for each bird
start_df = birds.groupby("tag-local-identifier")['geometry'].apply(list).apply(lambda x: x[0]).reset_index()
start_gdf = gpd.GeoDataFrame(start_df, geometry=start_df.geometry)
start_gdf.crs = {'init' :'epsg:4326'}

# Show first five rows of GeoDataFrame
start_gdf.head()

다음 코드 셀을 사용하여 각 새의 최종 위치를 포함하는 GeoDataFrame `end_gdf`를 만듭니다.
- 형식은 두 개의 열("tag-local-identifier" 및 "geometry")이 있는 `start_gdf`의 형식과 동일해야 합니다. 여기서 "geometry" 열은 Point 개체를 포함합니다.
- `end_gdf`의 CRS를 `{'init': 'epsg:4326'}`으로 설정합니다.

In [ ]:
# Your code here
end_gdf = end_df = birds.groupby("tag-local-identifier")['geometry'].apply(list).apply(lambda x: x[-1]).reset_index()
end_gdf = gpd.GeoDataFrame(end_df, geometry=end_df.geometry)
end_gdf.crs = {'init': 'epsg:4326'}

# Check your answer
#q_3.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_3.hint()
#q_3.solution()

### 4) Where does each bird start and end its journey? (Part 2)


위 질문의 GeoDataFrames(`path_gdf`, `start_gdf`, `end_gdf`)를 사용하여 단일 지도에서 모든 새의 경로를 시각화하세요. `americas` GeoDataFrame을 사용할 수도 있습니다.

In [ ]:
# Your code here
ax = americas.plot(figsize=(10, 10), color='white', linestyle=':', edgecolor='red')

start_gdf.plot(ax=ax, color='red',  markersize=30)
path_gdf.plot(ax=ax, cmap='tab20b', linestyle='-', linewidth=1, zorder=1)
end_gdf.plot(ax=ax, color='black', markersize=30)

# Uncomment to see a hint
#q_4.hint()

In [ ]:
# Get credit for your work after you have created a map
#q_4.check()

# Uncomment to see our solution (your code may look different!)
#q_4.solution()

### 5) Where are the protected areas in South America? (Part 1)

모든 새들이 남아메리카 어딘가에 있는 것처럼 보입니다. 그러나 그들은 보호 구역으로 가나요?

다음 코드 셀에서는 남미의 모든 보호 지역 위치를 포함하는 GeoDataFrame `protected_areas`를 생성합니다. 해당 shapefile은 파일 경로 `protected_filepath`에 있습니다.

In [ ]:
# Path of the shapefile to load
protected_filepath = "../input/geospatial-learn-course-data/SAPA_Aug2019-shapefile/SAPA_Aug2019-shapefile/SAPA_Aug2019-shapefile-polygons.shp"

# Your code here
protected_areas = protected_areas = gpd.read_file(protected_filepath)

# Check your answer
#q_5.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_5.hint()
#q_5.solution()

### 6) Where are the protected areas in South America? (Part 2)

'protected_areas' GeoDataFrame을 사용하여 남아메리카의 보호 지역 위치를 표시하는 플롯을 만듭니다. (_일부 보호 구역은 육지에 있고 다른 보호 구역은 바다에 있음을 알 수 있습니다._)

In [ ]:
# Country boundaries in South America
south_america = americas.loc[americas['continent']=='South America']

# Your code here: plot protected areas in South America
ax = south_america.plot(figsize=(10,10), color='white', edgecolor='red')
protected_areas.plot(ax=ax, alpha=0.4)

# Uncomment to see a hint
#q_6.hint()

In [ ]:
# Get credit for your work after you have created a map
#q_6.check()

# Uncomment to see our solution (your code may look different!)
#q_6.solution()

### 7) What percentage of South America is protected?

남아메리카에서 보호되는 비율을 확인하여 새에게 적합한 남아메리카의 비율을 확인하는 데 관심이 있습니다.

첫 번째 단계로 남아메리카의 모든 보호 구역(해양 지역 제외)의 총 면적을 계산합니다. 이렇게 하려면 총 면적과 총 해양 면적을 각각 평방 킬로미터 단위로 포함하는 "REP_AREA" 및 "REP_M_AREA" 열을 사용합니다.

변경 없이 아래 코드 셀을 실행합니다.

In [ ]:
P_Area = sum(protected_areas['REP_AREA']-protected_areas['REP_M_AREA'])
print("South America has {} square kilometers of protected areas.".format(P_Area))

그런 다음 계산을 완료하기 위해 `south_america` GeoDataFrame을 사용합니다.

In [ ]:
south_america.head()

다음 단계에 따라 남아메리카의 총 면적을 계산합니다.
- 각 폴리곤(CRS로 EPSG 3035 사용)의 'area' 속성을 사용하여 각 국가의 면적을 계산하고 결과를 합산합니다. 계산된 면적은 평방 미터 단위입니다.
- 답을 제곱킬로미터 단위로 변환합니다.

In [ ]:
# Your code here: Calculate the total area of South America (in square kilometers)
totalArea = totalArea = sum(south_america.geometry.to_crs(epsg=3035).area) / 10**6

# Check your answer
#q_7.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_7.hint()
#q_7.solution()

보호되는 남아메리카의 백분율을 계산하려면 아래 코드 셀을 실행하십시오.

In [ ]:
# What percentage of South America is protected?
percentage_protected = P_Area/totalArea
print('Approximately {}% of South America is protected.'.format(round(percentage_protected*100, 2)))

### 8) 남아메리카의 새들은 어디에 있습니까?

그렇다면 새들은 보호 구역에 있습니까?

모든 새, 남미에서 발견된 모든 위치를 보여주는 플롯을 만듭니다. 또한 남아메리카의 모든 보호 지역의 위치를 ​​표시합니다.

순수한 해양 지역(토지 구성 요소 없음)인 보호 지역을 제외하려면 "MARINE" 열을 사용할 수 있습니다(그리고 `protected_areas[protected_areas['MARINE']!='2']` `protected_areas` GeoDataFrame의 모든 행).

In [ ]:
# Your code here
ax = south_america.plot(figsize=(10,10), color='black', edgecolor='y')
protected_areas[protected_areas['MARINE']!='2'].plot(ax=ax, alpha=0.4, zorder=1)
birds[birds.geometry.y < 0].plot(ax=ax, color='r', alpha=0.6, markersize=10, zorder=2)
# Uncomment to see a hint
#q_8.hint()

In [ ]:
# Get credit for your work after you have created a map
#q_8.check()

# Uncomment to see our solution (your code may look different!)
#q_8.solution()

# Keep going

Create stunning **[interactive maps](https://www.kaggle.com/alexisbcook/interactive-maps)** with your geospatial data.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*